In [1]:
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_one = []
data_two = []
data_three = []
data_four = []
reviewdata_df = []

In [3]:
data_file = open('F:\yelp\yelp_academic_dataset_review.json')
length = 2225213  # I calculated length prior to this calculation
for i, line in enumerate(data_file):
    if i<(length/4):
        data_one.append(json.loads(line))
data_file.close()

In [4]:
df = pd.DataFrame(data_one)

In [5]:
data_main = []
with open('F:\yelp\yelp_academic_dataset_user.json') as f:
    for line in f:
        data_main.append(json.loads(line))

In [6]:
df_user = pd.DataFrame(data_main)

In [7]:
merged_df = pd.merge(df, df_user, on='user_id', how='inner')

In [8]:
merged_df.head(n=1)

,business_id,date,review_id,stars,text,type_x,user_id,votes_x,average_stars,compliments,elite,fans,friends,name,review_count,type_y,votes_y,yelping_since
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.08,"{u'note': 1, u'plain': 1, u'hot': 1, u'more': 1}",[],0,[],Jeffrey,60,user,"{u'funny': 25, u'useful': 166, u'cool': 12}",2009-05


In [9]:
merged_df.shape

(556303, 18)

In [10]:
merged_df.stars.value_counts(ascending=True)

2     51957
1     74210
3     77499
4    154479
5    198158
Name: stars, dtype: int64

In [11]:
data_business = []
with open('F:\yelp\yelp_academic_dataset_business.json') as f:
    for line in f:
        data_business.append(json.loads(line))

In [12]:
df_business = pd.DataFrame(data_business)

In [13]:
business_merged_df = pd.merge(merged_df, df_business, on='business_id', how='inner')

In [14]:
business_merged_df.drop(['neighborhoods', 'latitude', 'longitude', 'elite', 'yelping_since', 'compliments', 'full_address', 'city', 'state', 'categories', 'open', 'hours', 'attributes'], axis=1, inplace=True)

In [15]:
business_merged_df.head(n=1)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,friends,name_x,review_count_x,type_y,votes_y,name_y,review_count_y,stars_y,type
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.08,0,[],Jeffrey,60,user,"{u'funny': 25, u'useful': 166, u'cool': 12}",Mr Hoagie,4,4.5,business


In [16]:
for index, row in business_merged_df.head(n=50).iterrows():
    if len(row['friends']) != 0:
        print index

4
7
8
10
11
12
13
14
16
17
18
23
27
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [56]:
new_df = pd.DataFrame()
for index, row in business_merged_df.head(n=100).iterrows():
    if len(row['friends']) != 0:
        business_rating = row['stars_y']
        business_id = row['business_id']
        user_id = row['user_id']
        print "The business: " + str(business_id) + " has an avg rating of: " + str(business_rating)  
        user_rating = row['stars_x']
        print "User " + str(row['user_id']) +" rated " + str(business_id) + " with " + str(user_rating) + " stars. "
        count = 0
        friends = 0
        friends_who_rated = 0
        for fid in row['friends']:
#             if row['user_id'] == fid:
            friends += 1
            if any(business_merged_df[business_merged_df['user_id'] == fid]):
                for idx in business_merged_df[business_merged_df['user_id'] == fid].index:
#                    print business_merged_df.ix[idx]['user_id'] + "  =  " + fid   ##prints dat at idx, user & friend id r same
#                    print str(business_merged_df.ix[idx]['business_id']) + " at index "
                    if business_merged_df.ix[idx]['business_id'] == business_id:
                        print "Friend " + business_merged_df.ix[idx]['user_id'] + " rated " + business_merged_df.ix[idx]['business_id']
                        print "these many stars: " + business_merged_df.ix[idx]['stars_x'].astype(str)
                        count += business_merged_df.ix[idx]['stars_x']
                        friends_who_rated += 1
        print "Sum of total ratings given by friends: " + str(count)
        print "Total friends: " + str(friends)
        print "Total friends who rated the business: " + str(friends_who_rated)
        if friends_who_rated != 0:
            average = float(count/float(friends_who_rated))
            print "Friends average: " + str(average)
            data = pd.DataFrame({"business_id": [business_id], "business_stars": [business_rating], "user_stars": [user_rating], 
                                 "friends_avg_rating": [average], "user_id": [user_id], "friends_who_rated": [friends_who_rated],
                                 "total_friends": [friends]})
            new_df = new_df.append(data, ignore_index=True)
#                     else:
#                         print "no"
#             if any(business_merged_df['user_id'] == fid):
#                 print fid + " = " + business_merged_df['user_id']
#                 count += row['stars_x']
#         print "total friends' votes: %s" % count
#         print "total friends: %s" % friends
#                 count += row['stars_x']
#             print count


The business: WaHXyBwljbKNPmmJZn5j8Q has an avg rating of: 3.0
User 2jkE2AhFum7t3jHSngMZtg rated WaHXyBwljbKNPmmJZn5j8Q with 3 stars. 
Sum of total ratings given by friends: 0
Total friends: 12
Total friends who rated the business: 0
The business: u22QDAON1kk0wjoTliyftw has an avg rating of: 4.0
User 2l_VuWR75EUOxTarAwIK9w rated u22QDAON1kk0wjoTliyftw with 4 stars. 
Sum of total ratings given by friends: 0
Total friends: 2
Total friends who rated the business: 0
The business: u22QDAON1kk0wjoTliyftw has an avg rating of: 4.0
User 5vVOuKUHS1VaM-fsdS-gKw rated u22QDAON1kk0wjoTliyftw with 4 stars. 
Sum of total ratings given by friends: 0
Total friends: 7
Total friends who rated the business: 0
The business: 4ykgzzzGEWjMD5lwk1-l9A has an avg rating of: 4.5
User 7DeM4C66zVflYNrZn_Vv9g rated 4ykgzzzGEWjMD5lwk1-l9A with 2 stars. 
Sum of total ratings given by friends: 0
Total friends: 1
Total friends who rated the business: 0
The business: 4ykgzzzGEWjMD5lwk1-l9A has an avg rating of: 4.5
User

In [57]:
new_df

,business_id,business_stars,friends_avg_rating,friends_who_rated,total_friends,user_id,user_stars
0,2PfavOTufsPCRdYm-bFcpw,4.5,4.333333,6,55,l_szjd-ken3ma6oHDkTYXg,5
1,2PfavOTufsPCRdYm-bFcpw,4.5,4.575758,33,1444,nEYPahVwXGD2Pjvgkm7QqQ,5
2,2PfavOTufsPCRdYm-bFcpw,4.5,4.571429,14,230,bvu13GyOUwhEjPum2xjiqQ,4
3,2PfavOTufsPCRdYm-bFcpw,4.5,4.250000,4,79,T_wjLgPOPXry7Bea4MzoVQ,4
4,2PfavOTufsPCRdYm-bFcpw,4.5,4.575758,33,997,9cCTmiJ7hz35rHIdr8n9kA,4
5,2PfavOTufsPCRdYm-bFcpw,4.5,5.000000,1,2,lZpv9uSDKgyUxk1VPwyzDg,5
6,2PfavOTufsPCRdYm-bFcpw,4.5,4.500000,12,137,WzaaorVCmUTQvu4mScunNg,4
7,2PfavOTufsPCRdYm-bFcpw,4.5,4.555556,27,1138,9OZH1Ecw-qUkCW5MS0NefA,4
8,2PfavOTufsPCRdYm-bFcpw,4.5,4.000000,2,52,-xPKyCJiK9q1OFZ7GCZwBw,4
9,2PfavOTufsPCRdYm-bFcpw,4.5,4.333333,9,67,_vS9qNQvx8wVUOIkj4qfAg,4
